In [6]:
import mysql.connector
from mysql.connector import Error
import string
import json  # Import library JSON untuk menyimpan list sebagai string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# --- KONFIGURASI ---
# Ganti dengan detail koneksi database MySQL Anda
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root',
    'password': '',
    'database': 'twitter'
}

# Ganti dengan nama tabel dan kolom Anda
TABLE_NAME = 'trending' # GANTI INI
ID_COLUMN = 'id'
TEXT_COLUMN = 'isi_twit'

# Nama kolom untuk setiap langkah
STEP_COLUMNS = {
    'lower': 'lowercase',
    'no_punct': 'no_punctuation',
    'tokenized': 'tokenized',
    'no_stopwords': 'no_stopwords',
    'final': 'final_processed_text'
}

# --- FUNGSI PRA-PEMROSESAN ---
# Memuat stop words sekali saja untuk efisiensi
try:
    stop_words_id = set(stopwords.words('indonesian'))
    stop_words_en = set(stopwords.words('english'))
    stop_words_combined = stop_words_id.union(stop_words_en)
except LookupError:
    print("Stopwords corpus tidak ditemukan. Mengunduh...")
    nltk.download('stopwords')
    stop_words_id = set(stopwords.words('indonesian'))
    stop_words_en = set(stopwords.words('english'))
    stop_words_combined = stop_words_id.union(stop_words_en)


def process_and_get_steps(text):
    """
    Fungsi untuk melakukan pra-pemrosesan dan mengembalikan hasil setiap langkah dalam dictionary.
    """
    results = {}

    # Handle jika teks kosong atau bukan string
    if not isinstance(text, str):
        # Mengembalikan nilai default kosong untuk setiap langkah
        results[STEP_COLUMNS['lower']] = ''
        results[STEP_COLUMNS['no_punct']] = ''
        results[STEP_COLUMNS['tokenized']] = json.dumps([])
        results[STEP_COLUMNS['no_stopwords']] = json.dumps([])
        results[STEP_COLUMNS['final']] = ''
        return results

    # 1. Mengubah teks ke huruf kecil
    lower_text = text.lower()
    results[STEP_COLUMNS['lower']] = lower_text

    # 2. Menghapus tanda baca
    translator = str.maketrans('', '', string.punctuation)
    no_punct_text = lower_text.translate(translator)
    results[STEP_COLUMNS['no_punct']] = no_punct_text

    # 3. Tokenisasi kata
    tokens = word_tokenize(no_punct_text)
    # Simpan sebagai string JSON agar bisa disimpan di kolom TEXT
    results[STEP_COLUMNS['tokenized']] = json.dumps(tokens)

    # 4. Stop words removal
    clean_tokens = [word for word in tokens if word not in stop_words_combined and word.isalpha()]
    results[STEP_COLUMNS['no_stopwords']] = json.dumps(clean_tokens)

    # 5. Membuat string final yang sudah bersih
    final_text = ' '.join(clean_tokens)
    results[STEP_COLUMNS['final']] = final_text
    
    return results

# --- FUNGSI DATABASE ---
def setup_database_columns(conn):
    """Memeriksa dan menambahkan kolom yang diperlukan jika belum ada."""
    cursor = conn.cursor()
    print("Memeriksa struktur tabel...")
    try:
        for col_name in STEP_COLUMNS.values():
            cursor.execute(f"""
                SELECT COUNT(*) FROM information_schema.COLUMNS 
                WHERE TABLE_SCHEMA = '{DB_CONFIG['database']}' 
                AND TABLE_NAME = '{TABLE_NAME}' 
                AND COLUMN_NAME = '{col_name}'
            """)
            if cursor.fetchone()[0] == 0:
                print(f"Kolom '{col_name}' tidak ditemukan. Menambahkan...")
                cursor.execute(f"ALTER TABLE {TABLE_NAME} ADD COLUMN {col_name} TEXT NULL")
                print(f"Kolom '{col_name}' berhasil ditambahkan.")
        conn.commit()
    except Error as e:
        print(f"Error saat setup kolom: {e}")
        conn.rollback()
    finally:
        cursor.close()

# --- FUNGSI UTAMA ---
def main():
    """Fungsi utama untuk menghubungkan ke DB, memproses, dan menyimpan kembali."""
    conn = None
    try:
        conn = mysql.connector.connect(**DB_CONFIG)
        
        if conn.is_connected():
            print("Berhasil terhubung ke database MySQL.")
            
            # 1. Siapkan kolom-kolom di database
            setup_database_columns(conn)

            cursor = conn.cursor()
            
            # 2. Ambil data yang belum diproses (kita cek dari kolom final)
            query = f"""
                SELECT {ID_COLUMN}, {TEXT_COLUMN} 
                FROM {TABLE_NAME} 
                WHERE {TEXT_COLUMN} IS NOT NULL 
                AND {STEP_COLUMNS['final']} IS NULL
            """
            cursor.execute(query)
            rows = cursor.fetchall()
            
            if not rows:
                print("Tidak ada data baru untuk diproses.")
                return

            print(f"Ditemukan {len(rows)} baris data baru untuk diproses.")
            
            update_data = []

            # 3. Proses setiap baris
            for row in rows:
                tweet_id, original_text = row
                
                # Panggil fungsi pra-pemrosesan
                processed_steps = process_and_get_steps(original_text)
                
                # Siapkan data untuk update (pastikan urutannya benar)
                update_values = (
                    processed_steps[STEP_COLUMNS['lower']],
                    processed_steps[STEP_COLUMNS['no_punct']],
                    processed_steps[STEP_COLUMNS['tokenized']],
                    processed_steps[STEP_COLUMNS['no_stopwords']],
                    processed_steps[STEP_COLUMNS['final']],
                    tweet_id # ID untuk klausa WHERE
                )
                update_data.append(update_values)

            # 4. Simpan hasil kembali ke database menggunakan executemany (lebih cepat)
            if update_data:
                print("Menyimpan hasil ke database...")
                update_query = f"""
                    UPDATE {TABLE_NAME} SET 
                        {STEP_COLUMNS['lower']} = %s,
                        {STEP_COLUMNS['no_punct']} = %s,
                        {STEP_COLUMNS['tokenized']} = %s,
                        {STEP_COLUMNS['no_stopwords']} = %s,
                        {STEP_COLUMNS['final']} = %s
                    WHERE {ID_COLUMN} = %s
                """
                cursor.executemany(update_query, update_data)
                conn.commit()
                print(f"{cursor.rowcount} baris berhasil diperbarui di database.")

    except Error as e:
        print(f"Error saat menghubungkan atau memproses data: {e}")
    finally:
        if conn and conn.is_connected():
            conn.close()
            print("Koneksi MySQL ditutup.")
            
if __name__ == '__main__':
    # Memeriksa dan mengunduh resource NLTK yang diperlukan jika belum ada
    required_packages = ['punkt', 'stopwords', 'punkt_tab']
    for package in required_packages:
        try:
            # Coba cari paketnya
            nltk.data.find(f'tokenizers/{package.replace("_tab", "")}')
            if package == 'punkt_tab': # Pengecekan khusus untuk punkt_tab
                nltk.data.find('tokenizers/punkt_tab/english')
        except LookupError:
            # Jika tidak ditemukan, unduh
            print(f"Resource NLTK '{package}' tidak ditemukan. Mengunduh...")
            nltk.download(package)
    
    # Setelah semua paket dipastikan ada, jalankan fungsi utama
    main()

Resource NLTK 'stopwords' tidak ditemukan. Mengunduh...
Resource NLTK 'punkt_tab' tidak ditemukan. Mengunduh...


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


Berhasil terhubung ke database MySQL.
Memeriksa struktur tabel...
Ditemukan 80 baris data baru untuk diproses.
Menyimpan hasil ke database...
80 baris berhasil diperbarui di database.
Koneksi MySQL ditutup.
